In [1]:
!pip install --upgrade pandas numpy

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.


In [2]:
!pip install gensim

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [3]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.linear_model import LogisticRegression

In [4]:
def evaluate_model(clf, y_train, y_test, y_hat_train, y_hat_test):
    '''Print out the evaluation metrics for a given models predictions'''
    print(f'Classifier: {clf}', )
    print('-'*60)
    #plot_confusion_matrix(y_test,y_hat_test)
    print(f'test accuracy: {accuracy_score(y_test, y_hat_test)}')
    print(f'train accuracy: {accuracy_score(y_train, y_hat_train)}')
    print('-'*60)
    print('-'*60)
    print('Confusion Matrix:\n', pd.crosstab(y_test, y_hat_test, rownames=['Actual'], colnames=['Predicted'],margins = True))
    print('\ntest report:\n' + classification_report(y_test, y_hat_test))
    print('~'*60)
    return [accuracy_score(y_test, y_hat_test), classification_report(y_test, y_hat_test)]


## Loading data

In [6]:
df =pd.read_csv("mann_k.csv", sep="|")

In [7]:
df.head()

,file,Date,Subject,From,To,Body,Label
0,mann-k/deleted_items/1.,"Fri, 21 Dec 2001 15:59:54 -0800 (PST)",Out of Office AutoReply: Happy Holidays!,paula.rieker@enron.com,kay.mann@enron.com,Please note that I will be out of the office u...,deleted_items
1,mann-k/deleted_items/10.,"Wed, 26 Dec 2001 09:55:09 -0800 (PST)","FW: Dec. 24 -- BNA, Inc. Antitrust & Trade Reg...",sylvia.hu@enron.com,"clement.abrams@enron.com, jim.armogida@enron.c...","\n\n -----Original Message-----\nFrom: \t""BNA ...",deleted_items
2,mann-k/deleted_items/100.,"Tue, 20 Nov 2001 11:55:40 -0800 (PST)",RE:,kay.mann@enron.com,nmann@erac.com,Noyhing from Mom. She was going to leave at 8 ...,deleted_items
3,mann-k/deleted_items/101.,"Tue, 20 Nov 2001 11:34:00 -0800 (PST)",NaN,kay.mann@enron.com,alice.wright@enron.com,I had a last minute meeting come up. I'm on th...,deleted_items
4,mann-k/deleted_items/102.,"Tue, 20 Nov 2001 11:22:18 -0800 (PST)",NaN,kay.mann@enron.com,nmann@erac.com,How was lunch?\n--------------------------\nSe...,deleted_items


In [8]:
df['Label'].value_counts()

,count
Label,
discussion_threads,4949
sent,4440
deleted_items,172


## Preprocessing

In [9]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt_tab')
#from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [10]:
from nltk.corpus import stopwords
# Get the set of English stopwords
stop_words = set(stopwords.words('english'))

In [11]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(df):
  preprocessed_subject, preprocessed_content = [], []

  for index, row in df.iterrows():
      stem_sub, stem_cont = [], []

      if(not pd.isnull(row['Body'])):
        # Removing punctuations from the sentence
        sentence = re.sub(r'[^0-9A-Za-z]+', ' ', row['Body'])
        for word in word_tokenize(sentence):
            # Replace the word with stem word
            stem_word = (word.strip()).lower()
            if word not in stop_words:
              stem_word = lemmatizer.lemmatize(stem_word)
              stem_cont.append(stem_word)
      else:
        stem_cont.append(' ')

      if(not pd.isnull(row['Subject'])):
        # Removing punctuations from the sentence
        sentence = re.sub(r'[^0-9A-Za-z]+', ' ', row['Subject'])
        for word in word_tokenize(sentence):
            # Replace the word with stem word
            stem_word = (word.strip()).lower()
            if word not in stop_words:
              stem_word = lemmatizer.lemmatize(stem_word)
              stem_sub.append(stem_word)
      else:
        stem_sub.append(' ')

      preprocessed_subject.append(' '.join([word for word in stem_sub]))
      preprocessed_content.append(' '.join([word for word in stem_cont]))


  df['processed_subject'] = preprocessed_subject
  df['processed_body'] = preprocessed_content
  return df

In [12]:
email_df = preprocess_text(df)

In [13]:
email_df.head()

,file,Date,Subject,From,To,Body,Label,processed_subject,processed_body
0,mann-k/deleted_items/1.,"Fri, 21 Dec 2001 15:59:54 -0800 (PST)",Out of Office AutoReply: Happy Holidays!,paula.rieker@enron.com,kay.mann@enron.com,Please note that I will be out of the office u...,deleted_items,out office autoreply happy holiday,please note i office tuesday january 8 please ...
1,mann-k/deleted_items/10.,"Wed, 26 Dec 2001 09:55:09 -0800 (PST)","FW: Dec. 24 -- BNA, Inc. Antitrust & Trade Reg...",sylvia.hu@enron.com,"clement.abrams@enron.com, jim.armogida@enron.c...","\n\n -----Original Message-----\nFrom: \t""BNA ...",deleted_items,fw dec 24 bna inc antitrust trade regulation d...,original message from bna highlight bhighlig b...
2,mann-k/deleted_items/100.,"Tue, 20 Nov 2001 11:55:40 -0800 (PST)",RE:,kay.mann@enron.com,nmann@erac.com,Noyhing from Mom. She was going to leave at 8 ...,deleted_items,re,noyhing mom she going leave 8 9 we bean plus i...
3,mann-k/deleted_items/101.,"Tue, 20 Nov 2001 11:34:00 -0800 (PST)",NaN,kay.mann@enron.com,alice.wright@enron.com,I had a last minute meeting come up. I'm on th...,deleted_items,,i last minute meeting come i 6th floor the new...
4,mann-k/deleted_items/102.,"Tue, 20 Nov 2001 11:22:18 -0800 (PST)",NaN,kay.mann@enron.com,nmann@erac.com,How was lunch?\n--------------------------\nSe...,deleted_items,,how lunch sent blackberry wireless handheld ww...


In [14]:
email_df['SubjectTokens'] = email_df['processed_subject'].apply(lambda sentence: sentence.split())
email_df['ContentTokens'] = email_df['processed_body'].apply(lambda sentence: sentence.split())

In [15]:
email_df.head()

,file,Date,Subject,From,To,Body,Label,processed_subject,processed_body,SubjectTokens,ContentTokens
0,mann-k/deleted_items/1.,"Fri, 21 Dec 2001 15:59:54 -0800 (PST)",Out of Office AutoReply: Happy Holidays!,paula.rieker@enron.com,kay.mann@enron.com,Please note that I will be out of the office u...,deleted_items,out office autoreply happy holiday,please note i office tuesday january 8 please ...,"[out, office, autoreply, happy, holiday]","[please, note, i, office, tuesday, january, 8,..."
1,mann-k/deleted_items/10.,"Wed, 26 Dec 2001 09:55:09 -0800 (PST)","FW: Dec. 24 -- BNA, Inc. Antitrust & Trade Reg...",sylvia.hu@enron.com,"clement.abrams@enron.com, jim.armogida@enron.c...","\n\n -----Original Message-----\nFrom: \t""BNA ...",deleted_items,fw dec 24 bna inc antitrust trade regulation d...,original message from bna highlight bhighlig b...,"[fw, dec, 24, bna, inc, antitrust, trade, regu...","[original, message, from, bna, highlight, bhig..."
2,mann-k/deleted_items/100.,"Tue, 20 Nov 2001 11:55:40 -0800 (PST)",RE:,kay.mann@enron.com,nmann@erac.com,Noyhing from Mom. She was going to leave at 8 ...,deleted_items,re,noyhing mom she going leave 8 9 we bean plus i...,[re],"[noyhing, mom, she, going, leave, 8, 9, we, be..."
3,mann-k/deleted_items/101.,"Tue, 20 Nov 2001 11:34:00 -0800 (PST)",NaN,kay.mann@enron.com,alice.wright@enron.com,I had a last minute meeting come up. I'm on th...,deleted_items,,i last minute meeting come i 6th floor the new...,[],"[i, last, minute, meeting, come, i, 6th, floor..."
4,mann-k/deleted_items/102.,"Tue, 20 Nov 2001 11:22:18 -0800 (PST)",NaN,kay.mann@enron.com,nmann@erac.com,How was lunch?\n--------------------------\nSe...,deleted_items,,how lunch sent blackberry wireless handheld ww...,[],"[how, lunch, sent, blackberry, wireless, handh..."


## GloVe

In [16]:
email_df['processed_subject_body'] = email_df[['processed_subject','processed_body']].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)

In [17]:
email_df['tokens_subject_content'] =email_df['processed_subject_body'].map(lambda x: word_tokenize(x))

#new_df['tweet'] = email_df[['Title','subject_list']].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)
#new_df['target'] = email_df['Label']

In [18]:
# Get Labeled emails vocabulary
# put all tokens of each email in one set
total_vocabulary = set(word for processed_subject_body in email_df.tokens_subject_content for word in processed_subject_body)

len(total_vocabulary)
print('There are {} unique tokens in the dataset.'.format(len(total_vocabulary)))

There are 21995 unique tokens in the dataset.


In [19]:
# Get 100d GloVe embeddings for each word present in the vocabulary
glove = {}

with open(r'glove.6B.100d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [20]:
# Check length of glove dictionary relative to total words
print(f'{len(glove)} of {len(total_vocabulary)} \
    tokens in dataset matched in GloVe and vectorized!')

9667 of 21995     tokens in dataset matched in GloVe and vectorized!


In [21]:
class W2vVectorizer(object):
    """Class to generate mean word embeddings from word vectors. This class is
    quoted from Flatiron School Curriculum Learn.co Mod 4 appendix"""
    def __init__(self, w2v):
        # Takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])

    # Note: Even though it doesn't do anything,
    # it's required that this object implement a fit method or else
    # it can't be used in a scikit-learn pipeline
    def fit(self, X, y):
        return self

    def transform(self, X):
        # X should be a series of lists of tokens
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

In [22]:
# Instantiate a mean word embedding vectorizer using glove embeddings
vectorizer = W2vVectorizer(glove)

In [23]:
# Use vectorizer to transform tokenized data to vectors
email_glove = vectorizer.transform(email_df.tokens_subject_content)

In [24]:
# Check to see if email_glove is correct shape
# Should be # of observations (9561) by # word emb. dimension (100)
email_glove.shape

(9561, 100)

# Resampling Data by Smote Technique


In [25]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import  SMOTE
from sklearn.model_selection import GridSearchCV

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x79cb6c0ca3e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen() error


In [26]:
# Define X and y for the d2v models
#email_glove.shape
#y = email_df.Label
#y =
y = email_df.Label
X = email_glove

In [27]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y) #adding more users/non-users
print(pd.Series(y_resampled).value_counts())

X_train_smote, X_test_smote, Y_train_smote, Y_test_smote = train_test_split(X_resampled, y_resampled, random_state=0)

Label
deleted_items         4949
discussion_threads    4949
sent                  4949
Name: count, dtype: int64


## SVC Smote


In [28]:
# Instantiate classifier, fit, and predict on test data

svm_glove_smote = LinearSVC()
svm_glove_smote.fit(X_train_smote, Y_train_smote)
y_hat_test_smote = svm_glove_smote.predict(X_test_smote)

In [29]:
# Evaluate classifier performance

#results=evaluate_clf(y_test, y_hat_test)
svm_glove_smote_test_acc = round(accuracy_score(Y_test_smote, y_hat_test_smote), 2)
svm_glove_smote_test_acc

0.71

In [30]:
#Predictions
y_hat_train_smote= svm_glove_smote.predict(X_train_smote)
y_hat_test_smote = svm_glove_smote.predict(X_test_smote)

In [31]:
from sklearn import metrics

results=evaluate_model(svm_glove_smote, Y_train_smote, Y_test_smote, y_hat_train_smote, y_hat_test_smote)
glove_test_acc = round(accuracy_score(Y_test_smote,y_hat_test_smote),2)
glove_test_acc


Classifier: LinearSVC()
------------------------------------------------------------
test accuracy: 0.7125538793103449
train accuracy: 0.7119892231701841
------------------------------------------------------------
------------------------------------------------------------
Confusion Matrix:
 Predicted           deleted_items  discussion_threads  sent   All
Actual                                                           
deleted_items                1240                  24    17  1281
discussion_threads             94                 653   448  1195
sent                           61                 423   752  1236
All                          1395                1100  1217  3712

test report:
                    precision    recall  f1-score   support

     deleted_items       0.89      0.97      0.93      1281
discussion_threads       0.59      0.55      0.57      1195
              sent       0.62      0.61      0.61      1236

          accuracy                           0.71    

0.71

In [32]:
my_results = []
my_res = {
    "model": "LinearSVC SMOTE",
    "type": "Linear Classifier balanced by smote",
    " Test Accuracy": results[0],

}
my_results.append(my_res)

my_res_df = pd.DataFrame(my_results)
my_res_df

,model,type,Test Accuracy
0,LinearSVC SMOTE,Linear Classifier balanced by smote,0.712554


## LogisticRegresion Smote


In [33]:
# Import LogisticRegression from sklearn.linear_model
from sklearn.linear_model import LogisticRegression
# Instantiate classifier, fit, and predict on test data
lr_glove_smote = LogisticRegression()
lr_glove_smote.fit(X_train_smote, Y_train_smote)
y_hat_test_smote = lr_glove_smote.predict(X_test_smote)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
# Evaluate classifier performance

#results=evaluate_clf(y_test, y_hat_test)
lr_glove_smote_test_acc = round(accuracy_score(Y_test_smote, y_hat_test_smote), 2)
lr_glove_smote_test_acc

0.71

In [35]:
#Predictions
y_hat_train_smote= lr_glove_smote.predict(X_train_smote)
y_hat_test_smote = lr_glove_smote.predict(X_test_smote)

In [36]:

results=evaluate_model(lr_glove_smote, Y_train_smote, Y_test_smote, y_hat_train_smote, y_hat_test_smote)
lr_test_acc = round(accuracy_score(Y_test_smote,y_hat_test_smote),2)
lr_test_acc


Classifier: LogisticRegression()
------------------------------------------------------------
test accuracy: 0.705010775862069
train accuracy: 0.7093848226313426
------------------------------------------------------------
------------------------------------------------------------
Confusion Matrix:
 Predicted           deleted_items  discussion_threads  sent   All
Actual                                                           
deleted_items                1226                  38    17  1281
discussion_threads             89                 665   441  1195
sent                           56                 454   726  1236
All                          1371                1157  1184  3712

test report:
                    precision    recall  f1-score   support

     deleted_items       0.89      0.96      0.92      1281
discussion_threads       0.57      0.56      0.57      1195
              sent       0.61      0.59      0.60      1236

          accuracy                           

0.71

In [37]:
my_res = {
    "model": "LogisticRegresion SMOTE",
    "type": "Logistic Regresion balanced by smote",
    " Test Accuracy": results[0],

}
my_results.append(my_res)

my_res_df = pd.DataFrame(my_results)
my_res_df

,model,type,Test Accuracy
0,LinearSVC SMOTE,Linear Classifier balanced by smote,0.712554
1,LogisticRegresion SMOTE,Logistic Regresion balanced by smote,0.705011


## Random Forest Smote


In [38]:
# Instantiate classifier, fit, and predict on test data
rfc_glove_smote = RandomForestClassifier()
rfc_glove_smote.fit(X_train_smote, Y_train_smote)
y_hat_test_smote = rfc_glove_smote.predict(X_test_smote)

In [39]:
# Evaluate classifier performance

#results=evaluate_clf(y_test, y_hat_test)
rfc_glove_smote_test_acc = round(accuracy_score(Y_test_smote, y_hat_test_smote), 2)
rfc_glove_smote_test_acc

0.61

In [40]:
#Predictions
y_hat_train_smote= rfc_glove_smote.predict(X_train_smote)
y_hat_test_smote = rfc_glove_smote.predict(X_test_smote)

In [41]:
from sklearn import metrics

results=evaluate_model(rfc_glove_smote, Y_train_smote, Y_test_smote, y_hat_train_smote, y_hat_test_smote)
rfc_test_acc = round(accuracy_score(Y_test_smote,y_hat_test_smote),2)
rfc_test_acc


Classifier: RandomForestClassifier()
------------------------------------------------------------
test accuracy: 0.6136853448275862
train accuracy: 0.8494836102379884
------------------------------------------------------------
------------------------------------------------------------
Confusion Matrix:
 Predicted           deleted_items  discussion_threads  sent   All
Actual                                                           
deleted_items                1280                   1     0  1281
discussion_threads              6                 493   696  1195
sent                            0                 731   505  1236
All                          1286                1225  1201  3712

test report:
                    precision    recall  f1-score   support

     deleted_items       1.00      1.00      1.00      1281
discussion_threads       0.40      0.41      0.41      1195
              sent       0.42      0.41      0.41      1236

          accuracy                      

0.61

In [42]:

my_res = {
    "model": "Random Forest SMOTE",
    "type": "RFC balanced by smote",
    " Test Accuracy": results[0],

}
my_results.append(my_res)

my_res_df = pd.DataFrame(my_results)
my_res_df

,model,type,Test Accuracy
0,LinearSVC SMOTE,Linear Classifier balanced by smote,0.712554
1,LogisticRegresion SMOTE,Logistic Regresion balanced by smote,0.705011
2,Random Forest SMOTE,RFC balanced by smote,0.613685


## AdaBoost Classifier Smote


In [43]:
# Import AdaBoostClassifier from sklearn.ensemble
from sklearn.ensemble import AdaBoostClassifier
# Instantiate classifier, fit, and predict on test data
abc_glove_smote = AdaBoostClassifier()
abc_glove_smote.fit(X_train_smote, Y_train_smote)
y_hat_test_smote = abc_glove_smote.predict(X_test_smote)

In [44]:
# Evaluate classifier performance

#results=evaluate_clf(y_test, y_hat_test)
abc_glove_smote_test_acc = round(accuracy_score(Y_test_smote, y_hat_test_smote), 2)
abc_glove_smote_test_acc

0.64

In [45]:
#Predictions
y_hat_train_smote= abc_glove_smote.predict(X_train_smote)
y_hat_test_smote = abc_glove_smote.predict(X_test_smote)

In [46]:
from sklearn import metrics

results=evaluate_model(abc_glove_smote, Y_train_smote, Y_test_smote, y_hat_train_smote, y_hat_test_smote)
abc_test_acc = round(accuracy_score(Y_test_smote,y_hat_test_smote),2)
abc_test_acc


Classifier: AdaBoostClassifier()
------------------------------------------------------------
test accuracy: 0.6395474137931034
train accuracy: 0.6292770543331837
------------------------------------------------------------
------------------------------------------------------------
Confusion Matrix:
 Predicted           deleted_items  discussion_threads  sent   All
Actual                                                           
deleted_items                1111                  91    79  1281
discussion_threads            140                 574   481  1195
sent                          105                 442   689  1236
All                          1356                1107  1249  3712

test report:
                    precision    recall  f1-score   support

     deleted_items       0.82      0.87      0.84      1281
discussion_threads       0.52      0.48      0.50      1195
              sent       0.55      0.56      0.55      1236

          accuracy                          

0.64

In [47]:
my_res = {
    "model": "Ada Boost Clasifier SMOTE",
    "type": "ABC balanced by smote",
    " Test Accuracy": results[0],

}
my_results.append(my_res)

my_res_df = pd.DataFrame(my_results)
my_res_df

,model,type,Test Accuracy
0,LinearSVC SMOTE,Linear Classifier balanced by smote,0.712554
1,LogisticRegresion SMOTE,Logistic Regresion balanced by smote,0.705011
2,Random Forest SMOTE,RFC balanced by smote,0.613685
3,Ada Boost Clasifier SMOTE,ABC balanced by smote,0.639547
